In [1]:
# The full CNN code!
####################
import numpy as np
import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical

train_images = mnist.train_images() 
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Reshape the images.
train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

num_filters = 8
filter_size = 3
pool_size = 2

# Build the model.
model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
  MaxPooling2D(pool_size=pool_size),
  Flatten(),
  Dense(10, activation='softmax'),
])

# Compile the model.
model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

# Train the model.
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=3,
  validation_data=(test_images, to_categorical(test_labels)),
)

# Save the model to disk.
model.save_weights('cnn.h5')

# Load the model from disk later using:
# model.load_weights('cnn.h5')

# Predict on the first 5 test images.
predictions = model.predict(test_images[:5])

# Print our model's predictions.
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
print(test_labels[:5]) # [7, 2, 1, 0, 4]

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 7s 120us/step - loss: 0.3155 - acc: 0.9100 - val_loss: 0.1646 - val_acc: 0.9522
Epoch 2/3
60000/60000 [==============================] - 7s 117us/step - loss: 0.1502 - acc: 0.9565 - val_loss: 0.1285 - val_acc: 0.9619
Epoch 3/3
60000/60000 [==============================] - 7s 118us/step - loss: 0.1169 - acc: 0.9664 - val_loss: 0.1032 - val_acc: 0.9661
[7 2 1 0 4]
[7 2 1 0 4]


In [2]:
import numpy as np
import keras

class NeuralNetwork:
    def __init__(self):
        # number of hidden layers
        self.nHidden = args[0]

    # creating siamese network
    def CreateSiamese(self, *args):
        # the shape of the inputs
        input_shape = args[0]
        # creating a model variable
        model = keras.models.Sequential()
        # adding convolutional layers
        for layer in range (1, layers):
            # configuring first layer
            if layer == 0:
                configuration = Dense(nHidden, activation=NNArch[layer]['AF'], \
                                      kernel_initializer='random_normal', input_shape=input_shape)
            # configuring last layer
            elif layer == nLayers-1:
                configuration = Dense(nOutput, activation=NNArch[layer]['AF'], \
                                     kernel_initializer='random_normal')
            # configuring layers in between
            else:
                configuration = Dense(nHidden, activation=NNArch[layer]['AF'], \
                                     kernel_initializer='random_normal')
            model.add(configuration)
        
        # returning configured model
        return model

                
                # Feed Forward Neural Network
def CallKerasModel(NNArch, nLayers):
    classifier = Sequential()
    print("nHidden", nHidden)

    for layer in range(1, nLayers):
        if layer == 0:
            classifier.add(Dense(nHidden, activation=NNArch[layer]['AF'], \
                                 kernel_initializer='random_normal', input_dim=nFeatures))
        elif layer == nLayers-1:
            classifier.add(Dense(nOutput, activation=NNArch[layer]['AF'], \
                                 kernel_initializer='random_normal'))
        else:
            classifier.add(Dense(nHidden, activation=NNArch[layer]['AF'], \
                                 kernel_initializer='random_normal'))
    return classifier


In [3]:
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [5]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

import random
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
from keras.optimizers import RMSprop
from keras import backend as K

num_classes = 10
epochs = 20


def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)


def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)


def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    return Model(input, x)


def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))


# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
input_shape = x_train.shape[1:]

# create training+test positive and negative pairs
digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]
tr_pairs, tr_y = create_pairs(x_train, digit_indices)

digit_indices = [np.where(y_test == i)[0] for i in range(num_classes)]
te_pairs, te_y = create_pairs(x_test, digit_indices)

# network definition
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)

# train
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=128,
          epochs=epochs,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y))

# compute final accuracy on training and test sets
y_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(tr_y, y_pred)
y_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(te_y, y_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

Tensor("dropout_4/cond/Merge:0", shape=(?, 128), dtype=float32)
Train on 108400 samples, validate on 17820 samples
Epoch 1/20
108400/108400 [==============================] - 3s 29us/step - loss: 0.0944 - accuracy: 0.8899 - val_loss: 0.0406 - val_accuracy: 0.9594
Epoch 2/20
108400/108400 [==============================] - 3s 25us/step - loss: 0.0384 - accuracy: 0.9628 - val_loss: 0.0300 - val_accuracy: 0.9671
Epoch 3/20
108400/108400 [==============================] - 3s 26us/step - loss: 0.0272 - accuracy: 0.9723 - val_loss: 0.0271 - val_accuracy: 0.9691
Epoch 4/20
108400/108400 [==============================] - 3s 25us/step - loss: 0.0216 - accuracy: 0.9781 - val_loss: 0.0236 - val_accuracy: 0.9728
Epoch 5/20
108400/108400 [==============================] - 3s 26us/step - loss: 0.0187 - accuracy: 0.9802 - val_loss: 0.0240 - val_accuracy: 0.9718
Epoch 6/20
108400/108400 [==============================] - 3s 26us/step - loss: 0.0164 - accuracy: 0.9826 - val_loss: 0.0222 - val_accuracy

In [10]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

import random
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
from keras.optimizers import RMSprop
from keras import backend as K

def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    return Model(input, x)

create_base_network((100,100))

Tensor("flatten_7/Reshape:0", shape=(?, ?), dtype=float32)
Tensor("dense_14/Relu:0", shape=(?, 128), dtype=float32)


In [19]:
import os, sys
import numpy as np
import keras
from keras.models import Model
from keras.layers import Input, Dense

#from keras import backend as K

import time

'''
Class which has all necessary functions
'''
class Functions:
    # constructor
    def __init__(self):
        # passing the random seed
        seed = args[0]
        np.random.seed(seed)
    
    # Initialising Weights
    def GetWeights(self, *args):
        # the value from parameter file
        name = args[0]
        # random seed
        #seed = args[1]
        if name == 'norm':
            # random normal
            return keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=seed)
        elif name == 'unif':
            # random uniform
            return keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=seed)
        elif name == 'xnorm':
            # Xavier normal
            return keras.initializers.glorot_normal(seed=seed)
        elif name == 'xunif':
            # Xavier uniform
            return keras.initializers.glorot_uniform(seed=seed)
        elif name == 'hnorm':
            # He normal
            return keras.initializers.he_normal(seed=seed)
        elif name == 'hunif': 
            # He uniform
            return keras.initializers.he_uniform(seed=seed)
        else:
            print("Check your weights!")
            sys.exit()
            
    # Activation Functions
    def GetActivation(self, *args):
        # name of activation functions
        name = args[0]
        #
        x = args[1]
        if name == 'linear':
            # Linear
            return keras.activations.linear(x)
        elif name == 'exp':
            # Exponential
            return keras.activations.exponential(x)
        elif name == 'tanh':
            # Tanh
            return keras.activations.tanh(x)
        elif name == 'sigmoid':
            # Sigmoid
            return keras.activations.sigmoid(x)
        elif name == 'hsigmoid':
            # Hard Sigmoid
            return keras.activations.hard_sigmoid(x)
        elif name == 'softmax':
            # Softmax
            return keras.activations.softmax(x, axis=-1)
        elif name == 'softplus':
            # Softplus
            return keras.activations.softplus(x)
        elif name == 'softsign':
            # Softsign
            return keras.activations.softsign(x)
        elif name == 'relu':
            # ReLU
            return keras.activations.relu(x, alpha=0.0, max_value=None, threshold=0.0)
        elif name == 'elu':
            # eLU
            return keras.activations.elu(x, alpha=1.0)
        elif name == 'selu':
            # SeLU
            return keras.activations.selu(x)
        else:
            print('Check activation function!')
            sys.exit()
    
    # Regularization (L1, L2 and combined to avoid overfitting)
    def GetRegularizer(self, *args):
        name = args[0]
        
        if name == 'l1':
            return keras.regularizers.l1(0.)
        elif name == 'l2':
            return keras.regularizers.l2(0.)
        elif name == 'l1l2':
            return keras.regularizers.l1_l2(l1=0.01, l2=0.01)
        else:
            print('Check Regularization')
            sys.exit()
            
    # Optimization algorithms (to compile and run the model), 
    # i.e. various gradients methods
    def GetGradient(self, *args):
        # the name of the method
        name = args[0]
        # learning rate
        alpha = args[1]
        if name == 'sgd':
            # Stochastic Gradient Descent - includes momentum and support for Nesterov momentum
            return keras.optimizers.SGD(learning_rate=alpha, momentum=0.0, nesterov=False)
            # Nesterov
        elif name == 'nesterov':
            return keras.optimizers.SGD(learning_rate=alpha, momentum=0.0, nesterov=True)
        elif name == 'rmsprop':
            # RMSProp
            return keras.optimizers.RMSprop(learning_rate=alpha, rho=0.9)
        elif name == 'adagrad':
            # Adagrad
            return keras.optimizers.Adagrad(learning_rate=alpha)
        elif name == 'adadelta':
            # Adadelta
            return keras.optimizers.Adadelta(learning_rate=alpha, rho=0.95)
        elif name == 'adam':
            # Adam
            return keras.optimizers.Adam(learning_rate=alpha, beta_1=0.9, beta_2=0.999, amsgrad=False)
        elif name == 'adamax':
            # Adamax
            return keras.optimizers.Adamax(learning_rate=alpha, beta_1=0.9, beta_2=0.999)
        elif name == 'nadam':
            # Nadam
            return keras.optimizers.Nadam(learning_rate=alpha, beta_1=0.9, beta_2=0.999)
        else:
            print('Check optimization Methods!')
            sys.exit()
            
    # Getting Loss function
    def GetLoss(self, *args):
        name   = args[0]
        y_true = args[1]
        y_pred = args[2]
        # Getting the correct loss function
        if name == 'mse':
            # Mean Squared Error (MSE)
            return keras.losses.mean_squared_error(y_true, y_pred)
        elif name == 'mae':
            # Mean Absolute Error (MAE)
            return keras.losses.mean_absolute_error(y_true, y_pred)
        elif name == 'mape':
            # Mean Absolute Percentage Error (MAPE)
            return keras.losses.mean_absolute_percentage_error(y_true, y_pred)
        elif name == 'msle': 
            # Mean Squared Logarithmic Error (MSLE)
            return keras.losses.mean_squared_logarithmic_error(y_true, y_pred)
        elif name == 'hinge':
            # Hinge
            return keras.losses.hinge(y_true, y_pred)
        elif name == 'shinge':
            # Squared Hinge
            return keras.losses.squared_hinge(y_true, y_pred)
        elif name == 'chinge':
            # Categorical Hinge
            return keras.losses.categorical_hinge(y_true, y_pred)
        elif name == 'logcosh':
            # LogCosh
            return keras.losses.logcosh(y_true, y_pred)
        elif name == 'huber':
            # Huber Loss
            return keras.losses.huber_loss(y_true, y_pred, delta=1.0)
        elif name == 'categorical':
            # Categorical Cross Entropy
            return keras.losses.categorical_crossentropy(y_true, y_pred, from_logits=False, label_smoothing=0)
        elif name == 'sparse':
            # Sparse Categorical Cross Entropy
            return keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1)
        elif name == 'binary':
            # Binary Cross Entropy
            return keras.losses.binary_crossentropy(y_true, y_pred, from_logits=False, label_smoothing=0)
        elif name == 'kullback':
            # Kullback Leibler Divergence
            return keras.losses.kullback_leibler_divergence(y_true, y_pred)
        elif name == 'poisson':
            # Poisson
            return keras.losses.poisson(y_true, y_pred)
        elif name == 'proximity':
            # Cosine Proximity
            return keras.losses.cosine_proximity(y_true, y_pred, axis=-1)
        elif name == 'contrasive':
            '''Contrastive loss from Hadsell-et-al.'06
            http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
            '''
            margin = 1
            square_pred = keras.backend.square(y_pred)
            margin_square = keras.backend.square(keras.backend.maximum(margin - y_pred, 0))
            return keras.backend.mean(y_true * square_pred + (1 - y_true) * margin_square)
        else:
            print("Check Loss function!")
            sys.exit()

'''
Neural Network Class
'''
class NeuralNetwork:
    # Constructor - Getting Network Configuration
    def __init__(self, *args):
        # Network Type
        self.NNType = args[0]
        # Network Architecture
        self.NNArch = args[1]
        # Total Number of layers
        self.nLayers = args[2]
        # Neurons in Input Layer
        self.nInputNeurons = args[3]
        # Neurons in Hidden Layer
        self.nHiddenNeurons = args[4]
        # Neurons in Output Layer
        self.nOutputNeurons = args[5]
        # random seed
        seed = args[6]
        '''
        # Initializing object variable
        self.funcs = Functions(seed)
        # number of iterations for optimization algorithm
        self.epochs = args[6]
        # learning rate
        self.alpha = args[7]
        # regularisation (hyper) parameter
        self.lambd = args[8]
        # Data
        self.nInput = args[9]
        #self.nFeatures = 
        seed = args[10]
        # Batch Size
        self.BatchSize = args[11]
        # random seed, to make the same random number each time
        np.random.seed(seed)
        # optimization algorithm
        self.optimization = args[12]
        '''

        
        # Printing current network configuration
        """
        print((u'''
        =========================================== 
            Start {} Neural Network 
        =========================================== 
        No. of hidden layers:        {} 
        No. of input data:           {}
        No. of input neurons:        {} 
        No. of hidden neurons:       {} 
        No. of output neurons:       {} 
        Activ. Func in Hidden Layer: {} 
        Activ. Func in Output Layer: {} 
        No. of epochs to see:        {}
        Optimization Algorithm:      {}
        Learning Rate, \u03B1:            {} 
        Regularization param, \u03BB:     {} 
                      '''.format(self.NNType,
                                 self.nLayers-2,
                                 self.nInput,
                                 self.nInputNeurons, 
                                 self.nHiddenNeurons, 
                                 self.nOutputNeurons,
                                 self.NNArch[1]['AF'],
                                 self.NNArch[self.nLayers-1]['AF'],
                                 self.epochs,
                                 self.optimization,
                                 self.alpha,
                                 self.lambd)))
        """
    
    # Creating the model
    def BuildModel(self, *args):
        name = args[0]
        # shape of the inputs
        inputShape = args[1]
        # shape 
        
        # customizing our model
        inputs = Input(shape=inputShape)
        # adding (connecting) layers
        for layer in range(layers):
            # first layere
            if layer == 0:
                # getting activation function for the first layer
                activation = self.funcs.GetActivation(self.NNArch[layer]['AF'])
                hidden = Dense(2, activation = activation)(inputs)
            # last layer
            elif layer == layers:
                # getting activation function for the last layer
                activation = self.NNArch[layer]['AF']
                outputs = Dense(2, activation = activation)(hidden)
            # intermediate layers
            else:
                # getting activation function for the hidden layers
                activation = self.NNArch[layer]['AF']
                hidden = Dense(2, activation = activation)(hidden)
        # building the model
        model = Model(inputs = inputs, outputs = outputs)
        # returning the constructed model
        return model
    
'''
The main class of the program
'''
class Main:
    def __init__(self, *args):
        # getting the type of the program
        self.type = args[0]
        seed = args[1]
        # instantiating the Neural Network class variable
        self.network = NeuralNetwork(seed)
    
    def RunPipeline(self, *args):
        
        if self.type == 'siamese':
            print('Running siamese network')
        elif self.type == 'xgboost':
            print('Running XGBoost')
        elif self.type == 'rf':
            print('Running Random Forest')
            
if __name__ == '__main__':
    type = 'siamese'
    pipe = Main(type)
    pipe.RunPipeline()
        


Running siamese network
